In [1]:
VER = 1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd, numpy as np
import pickle, glob, gc

from collections import Counter
import itertools

# multiprocessing 
import psutil
N_CORES = psutil.cpu_count()     # Available CPU cores
print(f"N Cores : {N_CORES}")
from multiprocessing import Pool

N Cores : 8


In [9]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

def load_test(files):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(files)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [10]:
train = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/otto-validation/*_parquet/*')
print('train data has shape',train.shape)

train data has shape (171638757, 4)


In [14]:
train = train.groupby('aid').agg({'aid':'count','session':['nunique','count'],'type':['mean','std','min','max','sum']})
train.head()# sum,mean,min,max,count,std,nunique,cumcount

aid      session      type
    count  std nunique      mean
aid                             
0      38  0.0      30  0.000000
1      32  0.0      29  0.031250
2      16  0.0      15  0.000000
3    1525  0.0     852  0.083279
4     148  0.0     103  0.033784

In [15]:
train.tail()

aid      session      type
        count  std nunique      mean
aid                                 
1855598     7  0.0       7  0.000000
1855599     9  0.0       8  0.000000
1855600    70  0.0      44  0.071429
1855601    92  0.0      62  0.076087
1855602    15  0.0      14  0.000000

In [8]:
train.columns = ['item_item_count','item_user_count','item_buy_ratio']
train.head()

,item_item_count,item_user_count,item_buy_ratio
aid,,,
0,38,30,0.000000
1,32,29,0.031250
2,16,15,0.000000
3,1525,852,0.083279
4,148,103,0.033784


In [ ]:
def item_process(df):
    df = df.groupby('aid').agg({'aid':'count','session':'nunique','type':'mean'})
    df.columns = ['item_item_count','item_user_count','item_buy_ratio']
    return df

In [ ]:
train = item_process(train)
train.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/item/val_item{VER}.pqt')

In [ ]:
# FREE MEMORY
del train
_ = gc.collect()

In [ ]:
test = load_test('/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/original/*_parquet/*')
print('Test data has shape',test.shape)

Test data has shape (223644219, 4)


In [ ]:
test = item_process(test)
test.to_parquet(f'/content/drive/MyDrive/Colab Notebooks/kaggle/OTTO/dataset/candidate/item/test_item{VER}.pqt')